In [4]:
import pubchempy as pcp # Imports the library containing the infos on the molecules
from rdkit import Chem
from rdkit.Chem import Draw # Capasity to draw the molecule from the smile
from IPython.display import display # To have the molecule displayed in the notebook

def get_compound_info(molecule_name):
    try: # Try-except is to avoid errors to stop the code
        compounds = pcp.get_compounds(molecule_name, 'name') # Get compounds based on the molecule name

        if compounds:
            compound = compounds[0]  # Take the first result (assuming it's the correct one)
            iupac_name = compound.iupac_name
            smiles_code = compound.isomeric_smiles
            molar_mass = compound.molecular_weight
            atomic_formula = compound.molecular_formula
            # Retreaves the infos we want to display
            
            planar_molecule = Chem.MolFromSmiles(smiles_code)
            # Create a planar molecule object from the SMILES code
            
            stereo_centers = Chem.FindMolChiralCenters(planar_molecule, includeUnassigned=True)
            num_stereo_centers = len(stereo_centers)

            dict = {"iupac_name" : iupac_name,
                      "smiles_code" : smiles_code,
                      "molar_mass" : molar_mass,
                      "atomic_formula" : atomic_formula,
                      "planar_molecule" : planar_molecule,
                      "num_stereo_centers" : num_stereo_centers,
                      "stereo_centers" : stereo_centers}

            return dict
        else:
            return None
    except Exception as ex: # This part is just to avoid any error message to pop on the user's screen (would be terrible)
        return f"Error: {str(ex)}"



In [5]:
def label_stereogenic_centers(molecule, stereo_centers):
    for atom_idx, center_info in stereo_centers:
        atom = molecule.GetAtomWithIdx(atom_idx)
        neighbors = atom.GetNeighbors()
        
        # Calculate priorities for neighbors
        priorities = []
        for neighbor in neighbors:
            priority = neighbor.GetAtomicNum()  # Atomic number as priority
            priorities.append(priority)
        print(priorities)
        
        # Determine stereochemistry (R or S)
        if len(priorities) == 4: # No hydrogen
            # Sort priorities in descending order
            priorities.sort(reverse=True)
            if priorities[0]<priorities[1]<priorities[2]<priorities[3]:
                atom.SetProp("atomNote", "R")  # Rectus
            elif priorities[3]<priorities[2]<priorities[1]<priorities[0]:
                atom.SetProp("atomNote", "S")  # Sinister
            else:
                atom.SetProp("atomNote", "*")  # Unknown
        elif len(priorities) == 3: # One hydrogen
            # Sort priorities in descending order
            priorities.sort(reverse=True)
            if priorities[0]<priorities[1]<priorities[2]:
                atom.SetProp("atomNote", "R")  # Rectus
            elif priorities[2]<priorities[1]<priorities[0]:
                atom.SetProp("atomNote", "S")  # Sinister
            else:
                atom.SetProp("atomNote", "*")  # Unknown
        else:
            atom.SetProp("atomNote", "*")  # Unknown (not a stereocenter)


In [6]:
from rdkit import Chem

def get_chemical_groups(smiles):
    # Parse the SMILES string to obtain a molecule object
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return "Invalid SMILES string. Unable to parse molecule."

    # Get a list of atom indices for each chemical group
    chemical_groups = {}
    for atom in mol.GetAtoms():
        group = atom.GetSmarts()
        if group not in chemical_groups:
            chemical_groups[group] = []
        chemical_groups[group].append(atom.GetIdx())

    # Define priority order for chemical groups based on IUPAC nomenclature
    priority_order = {
        'CO': 0,                # Carboxylic acid
        'CC(=O)O': 1,           # Ester
        'C(=O)O': 1,            # Carboxylic acid derivative (e.g., acid chloride, anhydride)
        'NC(=O)': 2,            # Amide
        'CC(C)(C)O': 3,         # Tertiary alcohol
        'CCO': 3,               # Primary or secondary alcohol
        'C=C': 4,               # Alkene
        'CC#C': 5,              # Alkyne
        'CC#N': 6,              # Nitrile
        'CC(=O)N': 7,           # Amide (secondary)
        'C#N': 8,               # Cyanide
        'C(=O)N': 9,            # Amine
        'C#CC': 10,             # Alkynyl
        'C=CC': 11,             # Allyl
        'C#CC#C': 12,           # Diyne
        'COC': 13,              # Ether
        'CS': 14,               # Thiols
        'C(=O)Cl': 15,          # Acyl halide
        'C(=O)(OC)O': 16,       # Carbonate
        'COO': 17,              # Hydroxyperoxide
        'CC(=O)OCC': 18,        # Hemi-acetal
        'CC(=O)NCC': 19,        # Hemi-ketal
        'C1CCCC1': 20,          # Heterocycle
        'C(=O)(OC(=O))OC(=O)': 21,  # Carboxylic anhydride
        'C(=N)': 22,            # Imine
        'C(=O)(C)N': 23,        # Imide
        'N=N': 24,              # Azo
        'C(=[N+]=[N-])': 25,    # Azide
        'N=O': 26,              # Nitroso
        'C(O)N': 27,            # Oxyme
        'n1ccccc1': 28,         # Pyridyl
        'OC(=O)N': 29,          # Carbamate
        'CS': 30,               # Sulfide
        'CSSC': 31,             # Disulfide
        'CS(=O)': 32,           # Sulfoxide
        'CS(=O)(=O)': 33,       # Sulfone
        'COS(=O)(=O)': 34,      # Sulfonate ester
        'C#N': 35,              # Thiocyanate
        'C(=S)': 36,            # Thioketone
        'SC': 37,               # Thial
        'C(=S)(S)': 38,         # Thioester
        'C(=S)(=S)S': 39,       # Dithiocarboxylic acid
        'CP': 40,               # Phosphine
        'OC[Si]': 41            # Silyl ether
    }

    # Sort the chemical groups by priority according to IUPAC nomenclature
    sorted_groups = sorted(chemical_groups.keys(), key=lambda x: priority_order.get(x, float('inf')))

    # Return the sorted chemical groups
    return sorted_groups



molecule name: cafeine


IUPAC Name: 1,3,7-trimethylpurine-2,6-dione
SMILES Code: CN1C=NC2=C1C(=O)N(C(=O)N2C)C
Atomic Formula: C8H10N4O2
Molar Mass: 194.19 g/mol
Chemical groups: ['C', 'n', 'c', 'O']
Stereogenic Centers: 0


In [8]:

molecule_name = input("molecule name:")
result = get_compound_info(molecule_name)
if result:
    print(f"IUPAC Name: {result['iupac_name']}")
    print(f"SMILES Code: {result['smiles_code']}")
    print(f"Atomic Formula: {result['atomic_formula']}")
    print(f"Molar Mass: {result['molar_mass']} g/mol")
    print(f"Chemical groups: {get_chemical_groups(result['smiles_code'])}")
    print(f"Stereogenic Centers: {result['num_stereo_centers']}")

    
    label_stereogenic_centers(result["planar_molecule"], result["stereo_centers"])# Label stereogenic centers
    
    img = Draw.MolToImage(result['planar_molecule'])
    display(img)
else:
    print(f"No compound found for '{molecule_name}'")
    


In [8]:
import py3Dmol
from rdkit.Chem import AllChem

molecule_3D = Chem.AddHs(result['planar_molecule']) # Gets the 3D structure of the molecule
AllChem.EmbedMolecule(molecule_3D) # Makes that the molecule is not planar


# Helpful function from exercises 5
def drawit(m,p=None,confId=-1): # m is the molecule, p is the 3D object
        mb = Chem.MolToMolBlock(m,confId=confId) #Converts the molecule to a MolBlock (SDF format, the big table of coords) for visualization
        if p is None:
            p = py3Dmol.view(width=400,height=400)
        p.removeAllModels()
        p.addModel(mb,'sdf')
        p.setStyle({'stick': {}, 'sphere': {'colorscheme': 'Jmol', 'scale': 0.3}}) # Style of the molecule
        p.setBackgroundColor('#FFFFFF') # Background color (here white)
        p.zoomTo() # Zooming option
        return p.show()

drawit(molecule_3D)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.